<a href="https://colab.research.google.com/github/PhilipGunjari12/New-project-/blob/main/Fingerprint_based_cryptographic_key_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

gpkchvp_data_sets_path = kagglehub.dataset_download('gpkchvp/data-sets')
ashudy_thumbprint_model_path = kagglehub.notebook_output_download('ashudy/thumbprint-model')

print('Data source import complete.')


In [ ]:
!pip install datasketch
!pip install reedsolo

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from datasketch import MinHash
import reedsolo
import hashlib
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.hashes import SHA256
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.backends import default_backend


In [ ]:


dataset_path = "/kaggle/input/data-sets/74034_3_En_4_MOESM1_ESM/FVC2004/Dbs/DB2_B/"
print("Contents of dataset folder:", os.listdir(dataset_path))


In [ ]:
# Function to load fingerprint images
def load_images_from_folder(folder, img_size=(128, 128)):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
        if img is not None:
            img = cv2.resize(img, img_size)  # Resize image to (128x128)
            images.append(img)
    return np.array(images)


In [ ]:
# Load dataset
dataset_folder = "/kaggle/input/data-sets/74034_3_En_4_MOESM1_ESM/FVC2004/Dbs/DB2_B/"
X = load_images_from_folder(dataset_folder)
X = X.reshape(-1, 128, 128, 1)  # Reshape for CNN input
print(f"Loaded {len(X)} fingerprint images.")


In [ ]:
# Display sample images
if len(X) > 0:
    plt.figure(figsize=(10, 4))
    for i in range(min(5, len(X))):
        plt.subplot(1, 5, i+1)
        plt.imshow(X[i].reshape(128, 128), cmap='gray')
        plt.axis('off')
    plt.suptitle("Preprocessed Fingerprint Images")
    plt.show()
else:
    print("Error: No images found in the dataset!")


In [ ]:
# Define CNN Model with feature vector size = 64
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),

    Dense(128, activation='relu'),  # Feature extraction layer
    Dropout(0.5),

    Dense(64, activation='linear')  # Feature vector size changed to 64
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Display model summary
model.summary()

num_samples = X.shape[0]
feature_vector_size = 64

y = np.random.rand(num_samples, feature_vector_size)


In [ ]:
# Train CNN for 200 epochs
history = model.fit(X, y, batch_size=32, epochs=200, verbose=1)


In [ ]:
# Function to extract a feature vector from an image
def extract_feature_vector(image):
    img = image.reshape(1, 128, 128, 1)  # Reshape for CNN input
    feature_vector = model.predict(img)
    return feature_vector.flatten()

# Example: Extract feature vector from the first fingerprint
sample_feature_vector = extract_feature_vector(X[0])
print("Extracted Feature Vector:", sample_feature_vector)

In [ ]:
!pip install datasketch
!pip install reedsolo
!pip install cryptography

In [ ]:
# Function to generate a stable key using fuzzy extraction
def generate_stable_key(feature_vector, num_hashes=128):
    """Generates a stable cryptographic key from fingerprint data using fuzzy extraction."""

    # Step 1: Convert fingerprint to MinHash digest
    minhash = MinHash(num_perm=num_hashes)
    for val in feature_vector:
        minhash.update(int(val).to_bytes(2, byteorder='big'))  # Ensure conversion to int before bytes

    minhash_digest = minhash.digest()

    # Step 2: Apply Reed-Solomon encoding (adds redundancy for error correction)
    rs = reedsolo.RSCodec(10)  # Adding 10 error correction bytes
    encoded_digest = rs.encode(minhash_digest)

    # Step 3: Generate a cryptographic key using SHA-256
    stable_key = hashlib.sha256(encoded_digest).hexdigest()

    return stable_key, encoded_digest

# Example: Extract feature vector from a fingerprint image
sample_feature_vector = extract_feature_vector(X[0])

# Generate stable key
stable_key, stored_encoded_minhash = generate_stable_key(sample_feature_vector)
print("Generated Stable Key:", stable_key)


In [ ]:
#Convert Stable Key to ECC Private Key

def derive_ecc_key(stable_key):
    """Derives an ECC private key from the stable key using HKDF."""

    # Ensure stable_key is bytes (convert hex string to bytes)
    stable_key_bytes = bytes.fromhex(stable_key)  #Convert hex string to bytes

    # Use HKDF to derive a cryptographic key
    hkdf = HKDF(
        algorithm=hashes.SHA256(),
        length=32,  # ECC private keys are typically 32 bytes
        salt=None,
        info=b'fingerprint-derived-key',
        backend=default_backend()
    )

    private_key_bytes = hkdf.derive(stable_key_bytes)  # Now it's bytes

    # Convert to an ECC private key
    private_key = ec.derive_private_key(int.from_bytes(private_key_bytes, "big"), ec.SECP256R1(), default_backend())

    return private_key


In [ ]:
def encrypt_message(message, public_key):
    shared_secret = public_key.public_numbers().x.to_bytes(32, 'big')  # ECDH shared secret

    # AES-GCM Encryption
    iv = os.urandom(12)
    cipher = Cipher(algorithms.AES(shared_secret), modes.GCM(iv))
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(message.encode()) + encryptor.finalize()

    return iv, ciphertext, encryptor.tag

def decrypt_message(iv, ciphertext, tag, private_key):
    shared_secret = private_key.public_key().public_numbers().x.to_bytes(32, 'big')  # ECDH shared secret

    # AES-GCM Decryption
    cipher = Cipher(algorithms.AES(shared_secret), modes.GCM(iv, tag))
    decryptor = cipher.decryptor()
    decrypted_message = decryptor.update(ciphertext) + decryptor.finalize()

    return decrypted_message.decode()

#Testing
if len(X) > 0:
    feature_vector = extract_feature_vector(X[0])  # Extract feature vector
    stable_key, encoded_digest = generate_stable_key(feature_vector)  # Generate stable key
    private_key = derive_ecc_key(stable_key)  # Convert stable key to ECC private key
    public_key = private_key.public_key()  # Get ECC public key

    # Test Encryption & Decryption
    original_message = "Secure Biometric Encryption"
    iv, ciphertext, tag = encrypt_message(original_message, public_key)
    decrypted_message = decrypt_message(iv, ciphertext, tag, private_key)

    print("\nGenerated Stable Key (SHA-256):", stable_key.hex() if isinstance(stable_key, bytes) else stable_key)
    print("\nECC Private Key:", private_key.private_numbers().private_value)
    print("\nOriginal Message:", original_message)
    print("\nEncrypted Ciphertext:", ciphertext.hex())
    print("\nDecrypted Message:", decrypted_message)

else:
    print("Error: No fingerprint images found!")

In [ ]:
#from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import MobileNetV3Small  # or MobileNetV3Large
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input


In [ ]:
#Preprocess Fingerprint Dataset for MobileNet
def load_rgb_images_from_folder(folder, img_size=(128, 128)):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img_gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img_gray is not None:
            img_gray = cv2.resize(img_gray, img_size)
            img_rgb = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2RGB)
            images.append(img_rgb)
    return np.array(images)

dataset_path = "/kaggle/input/data-sets/74034_3_En_4_MOESM1_ESM/FVC2004/Dbs/DB1_A/"
X = load_rgb_images_from_folder(dataset_path)
X = X / 255.0  # Normalize
print(f"Loaded {len(X)} RGB images with shape {X.shape}")


In [ ]:
#Define MobileNetV3 Model for Feature Extraction
def build_mobilenetv3_feature_extractor(input_shape=(128, 128, 3)):
    #base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights=None)
    base_model = MobileNetV3Small(input_shape=(128, 128, 3), include_top=False, weights=None)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(64, activation='relu')(x)       # Custom feature vector
    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

model = build_mobilenetv3_feature_extractor()
model.summary()


In [ ]:
#Train with Dummy Labels
y = np.random.rand(len(X), 64)  # Feature vector size is now 64
model.fit(X, y, epochs=200, batch_size=16, verbose=0)

In [ ]:
#Feature Extraction Function
def extract_feature_vector(image_rgb):
    img = cv2.resize(image_rgb, (128, 128))
    img = img.reshape(1, 128, 128, 3) / 255.0
    return model.predict(img).flatten()
#Extracting feature vector from the first fingerprint
sample_feature_vector = extract_feature_vector(X[0])
print("Extracted Feature Vector:", sample_feature_vector)